# Session 9: Synthetic Data Generation and RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow, and use it to evaluate and iterate on a RAG pipeline with LangSmith!

**Learning Objectives:**
- Understand Ragas' knowledge graph-based synthetic data generation workflow
- Generate synthetic test sets with different query synthesizer types
- Load synthetic data into LangSmith for evaluation
- Evaluate a RAG chain using LangSmith evaluators
- Iterate on RAG pipeline parameters and measure the impact

## Table of Contents:

- **Breakout Room #1:** Synthetic Data Generation with Ragas
  - Task 1: Dependencies and API Keys
  - Task 2: Data Preparation and Knowledge Graph Construction
  - Task 3: Generating Synthetic Test Data
  - Question #1 & Question #2
  - 🏗️ Activity #1: Custom Query Distribution

- **Breakout Room #2:** RAG Evaluation with LangSmith
  - Task 4: LangSmith Dataset Setup
  - Task 5: Building a Basic RAG Chain
  - Task 6: Evaluating with LangSmith
  - Task 7: Modifying the Pipeline and Re-Evaluating
  - Question #3 & Question #4
  - 🏗️ Activity #2: Analyze Evaluation Results

---
# 🤝 Breakout Room #1
## Synthetic Data Generation with Ragas

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/martynakosciukiewicz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/martynakosciukiewicz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using two complementary guides — a Health & Wellness Guide covering exercise, nutrition, sleep, and stress management, and a Mental Health & Psychology Handbook covering mental health conditions, therapeutic approaches, resilience, and daily mental health practices. The topical overlap between documents helps RAGAS build rich cross-document relationships in the knowledge graph.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader("data/", glob="*.txt", loader_cls=TextLoader)
docs = loader.load()
print(f"Loaded {len(docs)} documents: {[d.metadata['source'] for d in docs]}")

Loaded 2 documents: ['data/MentalHealthGuide.txt', 'data/HealthWellnessGuide.txt']


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/martynakosciukiewicz/Documents/source/AIE9/09_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/martynakosciukiewicz/Documents/source/AIE9/09_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/martynakosciukiewicz/Documents/source/AIE9/09_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [9]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [10]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/7 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 9, relationships: 16)

We can save and load our knowledge graphs as follows.

In [11]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 9, relationships: 16)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

## ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### Answer:
They generate three different query scenarios:
- Single-Hop Specific Query - this query is easiest to answer as it can be addressed with a single retrieval (we need to get info from one doc source), hence "single-hop", meaning one step is required to answer this question
- Multi-Hop Specific Query - this one is harder to answer as it requires a few steps,i.e. the system needs to retrieve info from various places and figure out the connections between those to answer the query. Makes me think of multi-intent user queries in chatbots.
- Multi-Hop Abstract Query - similar to the previous one but it requires more reasoning and is less about facts but more about interpretations, hence "abstract"

Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What does the handbook say about the importanc...,[The Mental Health and Psychology Handbook A P...,The handbook explains that mental health encom...,single_hop_specifc_query_synthesizer
1,What is psychological resilience?,[PART 3: BUILDING RESILIENCE Chapter 7: What I...,Psychological resilience is the ability to ada...,single_hop_specifc_query_synthesizer
2,What is the main goal of COGNITVE BEHAVIORAL T...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Cognitive Behavioral Therapy focuses on identi...,single_hop_specifc_query_synthesizer
3,What is cognitive therapy and how does it rela...,[- Sleep restriction: Limiting time in bed to ...,Cognitive therapy involves addressing beliefs ...,single_hop_specifc_query_synthesizer
4,What are some effective stress reduction techn...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Effective stress management involves both imme...,single_hop_specifc_query_synthesizer
5,What are the signs that someone might need pro...,[<1-hop>\n\nThe Mental Health and Psychology H...,Signs indicating the need for professional men...,multi_hop_abstract_query_synthesizer
6,how sleep and CBT help mental health and sleep...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context explains that sleep is crucial for...,multi_hop_abstract_query_synthesizer
7,how nutriation and mental health and well-bein...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context shows that good nutrition like eat...,multi_hop_abstract_query_synthesizer
8,Can you explain how Chapter 13's focus on nutr...,[<1-hop>\n\n- Sleep restriction: Limiting time...,Certainly. Chapter 13 emphasizes the importanc...,multi_hop_specific_query_synthesizer
9,How does PART 4 relate to PART 2 in building r...,[<1-hop>\n\nPART 3: BUILDING RESILIENCE Chapte...,PART 4 discusses daily mental health practices...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are the recommended exercises and strateg...,[The Personal Wellness Guide A Comprehensive R...,The provided context does not include specific...,single_hop_specifc_query_synthesizer
1,What does Stage 2 of sleep involve in the slee...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Stage 2 involves a drop in body temperature an...,single_hop_specifc_query_synthesizer
2,What information does Chapter 18 cover regardi...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 18 discusses strategies to boost immun...,single_hop_specifc_query_synthesizer
3,How does the World Health Organization define ...,[The Mental Health and Psychology Handbook A P...,"According to the World Health Organization, me...",single_hop_specifc_query_synthesizer
4,how can exercise for common problems like lowe...,[<1-hop>\n\nThe Personal Wellness Guide A Comp...,The wellness guide explains that gentle exerci...,multi_hop_abstract_query_synthesizer
5,How can incorporating mindfulness and social c...,[<1-hop>\n\nhour before bed - No caffeine afte...,Incorporating mindfulness and social connectio...,multi_hop_abstract_query_synthesizer
6,How can improving face-to-face interactions an...,[<1-hop>\n\nhour before bed - No caffeine afte...,Improving face-to-face interactions by engagin...,multi_hop_abstract_query_synthesizer
7,How can I improve my emotional intelligence an...,[<1-hop>\n\nhour before bed - No caffeine afte...,To improve emotional intelligence and manage c...,multi_hop_abstract_query_synthesizer
8,how chapter 7 and 17 connect about sleep and h...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,"chapter 7 talks about sleep and recovery, expl...",multi_hop_specific_query_synthesizer
9,H0w c4n I bUild a he4lthy m0rn1ng r0utine (cha...,[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,To build a healthy morning routine that improv...,multi_hop_specific_query_synthesizer


## ❓ Question #2:

Ragas offers both an "unrolled" (manual) approach and an "abstracted" (automatic) approach to synthetic data generation. What are the trade-offs between these two approaches? When would you choose one over the other?

##### Answer:
- unrolled - allows for more control and customization as you configure it yourself. We can conntrol how to split documents, what info to extract (e.g. NER), how to establish relationships between the nodes, how to apply transformations to our data, how to synthesise queries and how to define personas. I would say this is more for mature projects where we understand well what we want to achieve, have a good grasp on our data, know our user well, etc. For example, when we have info about entities and properties between them (maybe a taxonomy or ontology), we may want to define the knowledge graph structure ourserlves. Also, when we have done user research or have a good understanding of our user base, we may want to define user personas manually. So it's more effort but more customised.
  
- abstracted - easier to implement, requires less effort but also gives less control. Feels like a way to start off, especially if we are prototyping and have only basic requirements and cursory understanding of our data and users. Less effor


---
## 🏗️ Activity #1: Custom Query Distribution

Modify the `query_distribution` to experiment with different ratios of query types.

### Requirements:
1. Create a custom query distribution with different weights than the default
2. Generate a new test set using your custom distribution
3. Compare the types of questions generated with the default distribution
4. Explain why you chose the weights you did

My understanding coming from working on customer service chatbots is that in a simple FAQ bot, for instance, we would have a higher proportion of single-hop simple queries, whereas in a more complex transactional bot or in a more refined domain, we may have more multi-hop queries (specific or abstract). Since we are using wellness data in this exercise I will simulate a use case that requires a more in-depth comparison of various health-related approaches. Hence, I want to increase the distribution of multi-hop queries. For example, users may ask "How does CBT compare to mindfulness techniques?" or "What are pros and cons of taking sleeping pills to improve sleep and tackle fatigue?" These would require retrieving info from various parts of the docs and synthesising them. I will still want to address single-hop queries cause someone can ask a simple "What is CBT?".

CONCLUSIONS (from the experiment implemented below):

The original testset, i.e. the one with the default query distribution, has more single-hop than multi-hop queries. Interestingly (and funnily), some of the queries are of dubious quality or use for an actual production-grade application, e.g. one query asks about the impact of nutrition on poor grammar, while multi-hop specific queries ask about the connections between various parts of the documents in the RAG pipeline (e.g. how does section A compare to section B). Depending on the actual use-case this may or may not be a good representation of actual user queries. So the quality leaves some room for improvement with this synthetically generated data:) However, I see that generating multi-hop specific queries (vs abstract) in the domain of mental health is a bit more difficult (the system would need to pick more concrete topics). On another note, some queries contain spelling mistakes which makes me wonder what caused that issue, perhaps something to do with transformations (e.g. row 7 "nutriation").

The new testset as expected has a higher proportion of multi-hop queries. Here, similarly to the previous testset, multi-hop specific queries are again mentioning specific sections in the docs. This testset also contains one query with spelling issues (row 3 "hygeene").

So the quality of queries in both testsets is the same, what changes is the number of query types, which is expected since I played around with this configuration. Another nice thing to explore would be testing different user personas to see how the query style and orientation changes.

In [29]:
# Define a custom query distribution with different weights
new_query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.3),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.35),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.35),
]

# Generate a new test set and compare with the default
new_testset = generator.generate(testset_size=10, query_distribution=new_query_distribution)

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [30]:
# Compare
# view the original testset
testset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What does the handbook say about the importanc...,[The Mental Health and Psychology Handbook A P...,The handbook explains that mental health encom...,single_hop_specifc_query_synthesizer
1,What is psychological resilience?,[PART 3: BUILDING RESILIENCE Chapter 7: What I...,Psychological resilience is the ability to ada...,single_hop_specifc_query_synthesizer
2,What is the main goal of COGNITVE BEHAVIORAL T...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Cognitive Behavioral Therapy focuses on identi...,single_hop_specifc_query_synthesizer
3,What is cognitive therapy and how does it rela...,[- Sleep restriction: Limiting time in bed to ...,Cognitive therapy involves addressing beliefs ...,single_hop_specifc_query_synthesizer
4,What are some effective stress reduction techn...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Effective stress management involves both imme...,single_hop_specifc_query_synthesizer
5,What are the signs that someone might need pro...,[<1-hop>\n\nThe Mental Health and Psychology H...,Signs indicating the need for professional men...,multi_hop_abstract_query_synthesizer
6,how sleep and CBT help mental health and sleep...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context explains that sleep is crucial for...,multi_hop_abstract_query_synthesizer
7,how nutriation and mental health and well-bein...,[<1-hop>\n\nPART 2: NUTRITION AND DIET Chapter...,The context shows that good nutrition like eat...,multi_hop_abstract_query_synthesizer
8,Can you explain how Chapter 13's focus on nutr...,[<1-hop>\n\n- Sleep restriction: Limiting time...,Certainly. Chapter 13 emphasizes the importanc...,multi_hop_specific_query_synthesizer
9,How does PART 4 relate to PART 2 in building r...,[<1-hop>\n\nPART 3: BUILDING RESILIENCE Chapte...,PART 4 discusses daily mental health practices...,multi_hop_specific_query_synthesizer


In [31]:
# view the new testset
new_testset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"As a wellness coach, how does mental health in...",[The Mental Health and Psychology Handbook A P...,"Mental health encompasses our emotional, psych...",single_hop_specifc_query_synthesizer
1,How does Dialectical Behavior Therapy (DBT) in...,[PART 2: THERAPEUTIC APPROACHES Chapter 4: Cog...,Dialectical Behavior Therapy (DBT) combines CB...,single_hop_specifc_query_synthesizer
2,How does cortisol relate to stress and mental ...,[Write letters to or from your future self Jou...,The context mentions cortisol as a stress horm...,single_hop_specifc_query_synthesizer
3,H0w do sleep hygeene practises affect sleep an...,[<1-hop>\n\nWrite letters to or from your futu...,"Sleep hygeine practises, such as setting a con...",multi_hop_abstract_query_synthesizer
4,"How do sleep hygiene practices, such as mainta...",[<1-hop>\n\nPART 3: SLEEP AND RECOVERY Chapter...,Sleep hygiene practices like maintaining a con...,multi_hop_abstract_query_synthesizer
5,how to recognize warning signs for mental heal...,[<1-hop>\n\nsocial interactions How to set and...,To recognize warning signs for mental health i...,multi_hop_abstract_query_synthesizer
6,how to build good habits and manage mental hea...,[<1-hop>\n\nsocial interactions How to set and...,"To build good habits and manage mental health,...",multi_hop_abstract_query_synthesizer
7,How do Chapters 8 and 19 together inform strat...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 8 discusses the science of habit forma...,multi_hop_specific_query_synthesizer
8,Whitch chapters cover the buildng of healthy h...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 13 discusses the science of habit form...,multi_hop_specific_query_synthesizer
9,How can practicing CBT and CBT-I help improve ...,[<1-hop>\n\nPART 2: THERAPEUTIC APPROACHES Cha...,Practicing Cognitive Behavioral Therapy (CBT) ...,multi_hop_specific_query_synthesizer


In [32]:
# display full content
import pandas as pd

#original testset
original_testset_df = testset.to_pandas()
with pd.option_context('display.max_colwidth', None):
    display(original_testset_df)

user_input  \
0                                                                                                                                                                                       What does the handbook say about the importance of mental health and how it impacts overall well-being?   
1                                                                                                                                                                                                                                                             What is psychological resilience?   
2                                                                                                                                                                                                                                         What is the main goal of COGNITVE BEHAVIORAL THERAPY?   
3                                                                                                                                                                                                                            What is cognitive therapy and how does it relate to sleep beliefs?   
4                                                                                                                                                                                                  What are some effective stress reduction techniques recommended for managing chronic stress?   
5                                                                                                      What are the signs that someone might need proffessional mental health support for conditions like anxiety or depression, and how do mental health conditions relate to physical health?   
6                                                                                                                                                                                                                                         how sleep and CBT help mental health and sleep cycles   
7                                                                                                                                                                                                                     how nutriation and mental health and well-being connect with poor grammar   
8                                                               Can you explain how Chapter 13's focus on nutrition and mental health connects with Chapter 5's guidance on meal planning, especially considering the themes of Chapter 13 and Chapter 5, despite the misspelling in the query?   
9                                                                                                                                                                                         How does PART 4 relate to PART 2 in building resilience and emotional intelligence for mental health?   
10  How does cognitive therapy, as part of cognitive behavioral therapy, help improve mental health by addressing negative thought patterns, and how is this approach supported by the themes of cognitive behavioral therapy and cognitive therapy in the context of mental health strategies?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [33]:
# new testset
with pd.option_context('display.max_colwidth', None):
    display(new_testset.to_pandas())

user_input  \
0                                                                                                                                                          As a wellness coach, how does mental health influence overall well-being?   
1                                                          How does Dialectical Behavior Therapy (DBT) incorporate mindfulness as a key skill set to help individuals manage their emotions and improve interpersonal effectiveness?   
2                                                                                                                                                                              How does cortisol relate to stress and mental health?   
3                                                                                                         H0w do sleep hygeene practises affect sleep and mental health, and how does sleep bidirectionally influence mental health?   
4   How do sleep hygiene practices, such as maintaining a consistent sleep schedule and creating a relaxing bedtime routine, support the sleep cycles and stages, including REM and non-REM sleep, to improve overall sleep quality?   
5                                                                                                                                     how to recognize warning signs for mental health issues and do self-care for mental well-being   
6                                                                                                                    how to build good habits and manage mental health with small steps and strategies from different professionals?   
7                                                                                            How do Chapters 8 and 19 together inform strategies for building healthy habits and maintaining work-life balance for overall wellness?   
8                                                                                                                            Whitch chapters cover the buildng of healthy habbits and how they relate to sleep and stress managment?   
9                                                             How can practicing CBT and CBT-I help improve mental health and sleep, especially considering the role of mindfulness and emotional regulation in building resilience?   
10                                How can CBT and CBT-I help improve mental health by addressing negative thoughts and sleep issues, especially considering the role of mindfulness and emotional regulation in building resilience?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

We'll need to provide our LangSmith API key, and set tracing to "true".

---
# 🤝 Breakout Room #2
## RAG Evaluation with LangSmith

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [34]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Use Case Synthetic Data - AIE9 - {uuid.uuid4()}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [35]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [36]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [39]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="use_case_rag"
)

In [40]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [41]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [42]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [43]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [44]:
rag_chain.invoke({"question" : "What are some recommended exercises for lower back pain?"})

'Recommended exercises for lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [45]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [46]:
from openevals.llm import create_llm_as_judge
from langsmith.evaluation import evaluate

# 1. QA Correctness (replaces LangChainStringEvaluator("qa"))
qa_evaluator = create_llm_as_judge(
    prompt="You are evaluating a QA system. Given the input, assess whether the prediction is correct.\n\nInput: {inputs}\nPrediction: {outputs}\nReference answer: {reference_outputs}\n\nIs the prediction correct? Return 1 if correct, 0 if incorrect.",
    feedback_key="qa",
    model="openai:gpt-4o" ,  # pass your LangChain chat model directly
)

# 2. Labeled Helpfulness (replaces LangChainStringEvaluator("labeled_criteria"))
labeled_helpfulness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "helpfulness: Is this submission helpful to the user, "
        "taking into account the correct reference answer?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n"
        "Reference answer: {reference_outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="helpfulness",
    model="openai:gpt-4o" ,
)

# 3. Dopeness (replaces LangChainStringEvaluator("criteria"))
dopeness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "dopeness: Is this response dope, lit, cool, or is it just a generic response?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="dopeness",
    model="openai:gpt-4o" ,
)

> **Describe what each evaluator is evaluating:**
>
> - `qa_evaluator`:
> - `labeled_helpfulness_evaluator`:
> - `dopeness_evaluator`:

## LangSmith Evaluation

In [47]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'stupendous-tooth-4' at:
https://smith.langchain.com/o/bea2e573-17c0-47b6-b892-bf2e11e9710b/datasets/3b2755ce-a592-45fa-b6f9-e606a14e2a6c/compare?selectedSessions=528c90d0-fbf3-428f-a995-b7e1675fccc3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.qa,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do chapters 8 and 20 connect about sleep a...,Chapter 8 discusses improving sleep quality th...,None,Chapter 8 explains how improving sleep quality...,True,True,True,3.717134,2cfca263-193b-483b-9bcf-10079cad50ed,019c6cae-a14c-7ce0-a031-f695be121fc2
1,How can understanding the science of habit for...,Based on the provided context from Chapters 13...,None,"According to Chapter 13, understanding the sci...",True,True,True,9.828473,50f23fdc-6feb-40aa-a28e-3415da7ee4c7,019c6cae-e0e8-7c02-964e-552039673ad7
2,How can CBT and CBT-I help improve mental heal...,CBT helps improve mental health by identifying...,None,Cognitive Behavioral Therapy (CBT) is an effec...,True,True,False,8.294609,b25acfdc-feb6-4d1c-b868-2c7730189dd5,019c6caf-4eba-78b2-8b65-6da285deaabd
3,how CBT and CBT-I help mental health and stress,Based on the provided context:\n\nCBT (Cogniti...,None,The context explains that CBT (Cognitive Behav...,True,True,False,4.370178,ddd4a520-8cd4-4abd-ab1b-757bba4300d9,019c6caf-9fb5-7c22-8b45-a7c9f7f7aef4
4,how mental health and CBT connect for good men...,Mental health and Cognitive Behavioral Therapy...,None,The context explains that mental health includ...,True,True,False,2.612927,dff98d7d-bfc0-4056-a607-2b0a1eac8c8b,019c6caf-e718-75b0-87a4-297442a8c6a2
5,how manage headach and digestion health support,To manage headaches naturally:\n\n- Drink wate...,None,The context explains that managing headaches i...,True,True,False,3.717081,1a69a91e-26d8-4581-a694-b0f35a77d7f6,019c6cb0-2155-7472-bbc2-26714625c90c
6,How does understanding the importance of sleep...,Understanding the importance of sleep for heal...,None,The context emphasizes that sleep is crucial f...,True,True,True,4.095617,fbb357c1-4495-4fa5-8f9e-552c844122db,019c6cb0-5882-7fd3-94a9-6676a2f74e83
7,How factors like sleep and physical health inf...,Factors like sleep and physical health have si...,None,The context explains that mental health is inf...,True,True,True,6.448645,ee98d25f-defb-492c-b550-04c6b82d32d1,019c6cb0-a1b7-7723-bb2b-3bcfb2083288
8,"How does FOMO influence digital mental health,...",Based on the provided context:\n\nFOMO (Fear o...,None,Fear of missing out (FOMO) drives compulsive c...,True,True,True,3.874948,f6d8da2c-ca74-4196-926d-dc9249517383,019c6cb0-fc2a-71d3-bb8d-12c4377c1140
9,How does serotonin relate to mental health acc...,"According to the provided context, serotonin i...",None,The context mentions that exercise increases s...,True,True,False,1.839127,3048f0d3-f45e-4c61-88a8-934888a3c4b3,019c6cb1-44ed-7b82-8b1b-6839c547f150


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [48]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [49]:
rag_documents = docs

In [50]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

## ❓ Question #3:

Why would modifying our chunk size modify the performance of our application?

##### Answer:
Chunk size affects how much context the model gets for grounding its responses in during retrieval. For example, if we choose larger chunk size, the model gets larger context per chunk for answering the user query. This may improve the performance but also worsen it - if we give too much info  in a chunk, like whole paragraphs, we may be introducing too much unrelated info for the model to generate a response. If a chunk is too small then the model may get fragmented info from the docs and the responses may not be coherent.

In [51]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## ❓ Question #4:

Why would modifying our embedding model modify the performance of our application?

##### Answer:
By changing the embedding model we affect the quality of the vector representation. This in turn affect the mapping of the user query to the info in our data store (chunks) and ultimately affects the quality of generated responses. Technically, a larger model would produce better vector representations, however this choice should be made based on project requirements (weighing model capabilities per specific domain vs costs etc.).

In [52]:
from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [53]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [54]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [55]:
dopeness_rag_chain.invoke({"question" : "How can I improve my sleep quality?"})

'Alright, ready to level up your sleep game and crash like a legend? Here’s the ultimate cheat code to unlocking top-tier sleep quality straight from the sleep sages:\n\n**1. Lock in a consistent sleep schedule, no matter if it’s weekday grind or weekend vibes.** Your body’s internal clock thrives on routine — it’s like telling your brain, “Yo, it’s time to power down.”\n\n**2. Craft a bedtime ritual that’s pure chill: dive into a book, do some gentle stretching, or soak in a warm bath.** That signals your nervous system to kick back and prepare for dreamland.\n\n**3. Transform your sleep lair:** keep it cool (65-68°F / 18-20°C), pitch-black with blackout curtains or a mask, and humming quietly with white noise or earplugs if needed. Comfort counts — invest in a mattress and pillows worthy of royalty.\n\n**4. Cut the caffeine after 2 PM, and dodge heavy meals or booze before hitting the hay.** These sneaky disruptors mess with your sleep cycle like uninvited party crashers.\n\n**5. Dit

Finally, we can evaluate the new chain on the same test set!

In [56]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'crazy-scene-91' at:
https://smith.langchain.com/o/bea2e573-17c0-47b6-b892-bf2e11e9710b/datasets/3b2755ce-a592-45fa-b6f9-e606a14e2a6c/compare?selectedSessions=f270feb4-00ac-4d2e-a769-7aff5e665933




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.qa,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do chapters 8 and 20 connect about sleep a...,"Yo, let’s break down this epic mind-body link ...",None,Chapter 8 explains how improving sleep quality...,True,True,True,5.664565,2cfca263-193b-483b-9bcf-10079cad50ed,019c6cc7-a7c4-77a0-b1a2-e8f0377322e9
1,How can understanding the science of habit for...,"Alright, buckle up for a wellness wisdom drop ...",None,"According to Chapter 13, understanding the sci...",1,True,True,11.893508,50f23fdc-6feb-40aa-a28e-3415da7ee4c7,019c6cc7-f2a4-73b0-9c02-64ccf27ad062
2,How can CBT and CBT-I help improve mental heal...,"Alright, let’s crank up the cool factor on thi...",None,Cognitive Behavioral Therapy (CBT) is an effec...,True,True,True,8.811408,b25acfdc-feb6-4d1c-b868-2c7730189dd5,019c6cc8-6b15-7d61-b54b-630a38e37f6d
3,how CBT and CBT-I help mental health and stress,"Alright, buckle up for a mind-blowing dive int...",None,The context explains that CBT (Cognitive Behav...,True,True,True,8.436004,ddd4a520-8cd4-4abd-ab1b-757bba4300d9,019c6cc8-e9e8-7ee1-9574-39e0cbbc5e07
4,how mental health and CBT connect for good men...,"Alright, let’s crank up the mental health mojo...",None,The context explains that mental health includ...,True,True,True,5.162904,dff98d7d-bfc0-4056-a607-2b0a1eac8c8b,019c6cc9-54ef-7511-8616-efff7d7c090d
5,how manage headach and digestion health support,"Alright, here’s the ultimate, no-nonsense, rad...",None,The context explains that managing headaches i...,True,True,True,7.242476,1a69a91e-26d8-4581-a694-b0f35a77d7f6,019c6cc9-9396-7362-b01f-0e6a9b5432d0
6,How does understanding the importance of sleep...,"Alright, strap in because we’re about to conne...",None,The context emphasizes that sleep is crucial f...,True,True,True,7.935461,fbb357c1-4495-4fa5-8f9e-552c844122db,019c6cc9-dfdc-7253-aaa7-51b31cc35478
7,How factors like sleep and physical health inf...,"Alright, let’s plug into this mental health ma...",None,The context explains that mental health is inf...,True,True,True,9.871389,ee98d25f-defb-492c-b550-04c6b82d32d1,019c6cca-363d-71d3-8cb9-809221e14296
8,"How does FOMO influence digital mental health,...","Yo, FOMO—the Fear of Missing Out—is like a dig...",None,Fear of missing out (FOMO) drives compulsive c...,True,True,True,4.914415,f6d8da2c-ca74-4196-926d-dc9249517383,019c6cca-8451-73c0-80e2-3a268f75ede0
9,How does serotonin relate to mental health acc...,"Alright, let’s crank this up to ultra-rad brai...",None,The context mentions that exercise increases s...,False,False,True,3.444676,3048f0d3-f45e-4c61-88a8-934888a3c4b3,019c6cca-ca3f-75c0-828c-610aa9248e35


---
## 🏗️ Activity #2: Analyze Evaluation Results

Provide a screenshot of the difference between the two chains in LangSmith, and explain why you believe certain metrics changed in certain ways.

##### Answer:

Chain 1: chunk size 500, embedding model: text-embedding-3-small
Chain 2: chunk size 1000, embedding mode: text-embedding-3-large

The responses are more dope! :D the other metrics are not significantly improved nor worsened. However, the latency increases (almost doubles). Hence, I would measure the improvements in the form of dope responses versu the minimal change in metrics and higher latency. Of course, if the user experience relies on the dope answers, then maybe it's worth the wait! :D

[Click to see chain 1](./chain1.png)

[Click to see chain 2](./chain2.png)

---
## Summary

In this session, we:

1. **Generated synthetic test data** using Ragas' knowledge graph-based approach
2. **Explored query synthesizers** for creating diverse question types
3. **Loaded synthetic data** into a LangSmith dataset for evaluation
4. **Built and evaluated a RAG chain** using LangSmith evaluators
5. **Iterated on the pipeline** by modifying chunk size, embedding model, and prompt — then measured the impact

### Key Takeaways:

- **Synthetic data generation** is critical for early iteration — it provides high-quality signal without manually creating test data
- **LangSmith evaluators** enable systematic comparison of pipeline versions
- **Small changes matter** — chunk size, embedding model, and prompt modifications can significantly affect evaluation scores